In [2]:
import zipfile

# Download zip file of pizza_steak images
!wget https://storage.googleapis.com/ztm_tf_course/food_vision/pizza_steak.zip

# Unzip the downloaded file
zip_ref = zipfile.ZipFile("pizza_steak.zip", "r")
zip_ref.extractall()
zip_ref.close()

--2023-11-02 01:57:12--  https://storage.googleapis.com/ztm_tf_course/food_vision/pizza_steak.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 74.125.204.207, 64.233.187.207, 64.233.188.207, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|74.125.204.207|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 109540975 (104M) [application/zip]
Saving to: ‘pizza_steak.zip’

pizza_steak.zip     100%[===================>] 104.47M  26.7MB/s    in 4.9s    

2023-11-02 01:57:17 (21.3 MB/s) - ‘pizza_steak.zip’ saved [109540975/109540975]



In [3]:
import os

for dirpath, dirnames, filenames in os.walk("pizza_steak"):
  print(f"There are {len(dirnames)} directories and {len(filenames)} images in '{dirpath}'.")

There are 2 directories and 0 images in 'pizza_steak'.
There are 2 directories and 0 images in 'pizza_steak/test'.
There are 0 directories and 250 images in 'pizza_steak/test/steak'.
There are 0 directories and 250 images in 'pizza_steak/test/pizza'.
There are 2 directories and 0 images in 'pizza_steak/train'.
There are 0 directories and 750 images in 'pizza_steak/train/steak'.
There are 0 directories and 750 images in 'pizza_steak/train/pizza'.


In [4]:
num_steak_images_train = len(os.listdir("pizza_steak/train/steak"))

num_steak_images_train

750

In [3]:
# Get the class names (programmatically, this is much more helpful with a longer list of classes)
import pathlib
import numpy as np
data_dir = pathlib.Path("pizza_steak/train/") # turn our training path into a Python path
class_names = np.array(sorted([item.name for item in data_dir.glob('*')])) # created a list of class_names from the subdirectories
print(class_names)

['pizza' 'steak']


In [8]:
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import random

def view_random_image(target_dir, target_class):
  # Setup target directory (we'll view images from here)
  target_folder = target_dir+target_class

  # Get a random image path
  random_image = random.sample(os.listdir(target_folder), 1)

  # Read in the image and plot it using matplotlib
  img = mpimg.imread(target_folder + "/" + random_image[0])
  plt.imshow(img)
  plt.title(target_class)
  plt.axis("off")

  print(f"Image shape: {img.shape}") # show the shape of the image

  return img


In [ ]:
img = view_random_image(target_dir="pizza_steak/train/",
                        target_class="steak")

In [ ]:
import tensorflow as tf
tf.constant(img)

In [ ]:
img.shape # width, height, colour channels

In [ ]:
# get all the balue in 0 & 1
img / 255.

In [4]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator

tf.random.set_seed(42)
# Preprocess data (get all of the pixel values between 1 and 0, also called scaling/normalization)
train_datagen = ImageDataGenerator(rescale=1./255)
valid_datagen = ImageDataGenerator(rescale=1./255)

# set a path from our current directories
train_dir = "pizza_steak/train/"
test_dir = "pizza_steak/test/"

train_data = train_datagen.flow_from_directory(directory=train_dir,
                                               batch_size=32,
                                               target_size = (244,244),
                                               class_mode="binary",
                                               seed=42)

valid_data = valid_datagen.flow_from_directory(directory=test_dir,
                                               batch_size=32,
                                               target_size = (244,244),
                                               class_mode="binary",
                                               seed=42)
# Import data from directories and turn it into batches
train_data = train_datagen.flow_from_directory(directory=train_dir,
                                               batch_size=32, # number of images to process at a time
                                               target_size=(224, 224), # convert all images to be 224 x 224
                                               class_mode="binary", # type of problem we're working on
                                               seed=42)

valid_data = valid_datagen.flow_from_directory(directory=test_dir,
                                               batch_size=32,
                                               target_size=(224, 224),
                                               class_mode="binary",
                                               seed=42)



# build cnn model
model_1 = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(filters=10, kernel_size=3,
                           activation='relu',
                           input_shape=(224,224,3)),
    tf.keras.layers.Conv2D(10, 3, activation='relu'),
    tf.keras.layers.MaxPool2D(pool_size=2, padding="valid"),
    tf.keras.layers.Conv2D(10,3, activation='relu'),
    tf.keras.layers.Conv2D(10,3, activation='relu'),
    tf.keras.layers.MaxPool2D(2),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

model_1.compile(loss="binary_crossentropy",
                optimizer=tf.keras.optimizers.Adam(),
                metrics=["accuracy"])


# label and data created for us using flow_from_directory
history_1 = model_1.fit(train_data,
                        epochs=5,
                        steps_per_epoch=len(train_data), # total img / batch_size = total steps for machine to know and memory calculation for storing
                        validation_data=valid_data,
                        validation_steps=len(valid_data))

Found 1500 images belonging to 2 classes.
Found 500 images belonging to 2 classes.
Found 1500 images belonging to 2 classes.
Found 500 images belonging to 2 classes.
Epoch 1/5
47/47 [==============================] - 19s 157ms/step - loss: 0.5900 - accuracy: 0.6793 - val_loss: 0.4407 - val_accuracy: 0.8040
Epoch 2/5
47/47 [==============================] - 6s 128ms/step - loss: 0.4442 - accuracy: 0.8013 - val_loss: 0.3756 - val_accuracy: 0.8480
Epoch 3/5
47/47 [==============================] - 6s 127ms/step - loss: 0.4148 - accuracy: 0.8140 - val_loss: 0.4109 - val_accuracy: 0.8080
Epoch 4/5
47/47 [==============================] - 6s 131ms/step - loss: 0.3672 - accuracy: 0.8540 - val_loss: 0.3199 - val_accuracy: 0.8840
Epoch 5/5
47/47 [==============================] - 5s 114ms/step - loss: 0.3183 - accuracy: 0.8667 - val_loss: 0.3415 - val_accuracy: 0.8540


## Update in tensorflow

below you can see we have used ImageDataGenerator which has been updated to image_dataset_from_directory of tf.keras.utils.

The main differences between them are:

*   rescaling & data loading:
    

  1.   In `ImageDataGenerator` we rescaled imag then applied flow_from_directory, we used **target_size** and **class_mode**
  2.   In `image_dataset_from_directory` we got the data from directory using **image_size** and **label_mode** which are used same as *target_size* and *class_mode*. here, to rescale image we have added new layer in Sequential Model:

```
tf.keras.layers.Rescaling(1./255,input_shape=(224,224,3)),
```






In [10]:
from tensorflow.keras.utils import image_dataset_from_directory

tf.random.set_seed(42)


# set a path from our current directories
train_dir = "pizza_steak/train/"
test_dir = "pizza_steak/test/"

train_data = image_dataset_from_directory(directory=train_dir,
                                               batch_size=32,
                                               image_size = (224,224),
                                               label_mode="binary",
                                               seed=42)

valid_data = image_dataset_from_directory(directory=test_dir,
                                               batch_size=32,
                                               image_size = (224,224),
                                               label_mode="binary",
                                               seed=42)

# build cnn model
model_1 = tf.keras.models.Sequential([

    tf.keras.layers.Rescaling(1./255,input_shape=(224,224,3)),
    tf.keras.layers.Conv2D(filters=10, kernel_size=3, activation='relu'),
    tf.keras.layers.Conv2D(10, 3, activation='relu'),
    tf.keras.layers.MaxPool2D(pool_size=2, padding="valid"),
    tf.keras.layers.Conv2D(10,3, activation='relu'),
    tf.keras.layers.Conv2D(10,3, activation='relu'),
    tf.keras.layers.MaxPool2D(2),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

model_1.compile(loss="binary_crossentropy",
                optimizer=tf.keras.optimizers.Adam(),
                metrics=["accuracy"])


# label and data created for us using flow_from_directory
history_1 = model_1.fit(train_data,
                        epochs=5,
                        steps_per_epoch=len(train_data), # total img / batch_size = total steps for machine to know and memory calculation for storing
                        validation_data=valid_data,
                        validation_steps=len(valid_data))

Found 1500 files belonging to 2 classes.
Found 500 files belonging to 2 classes.
Epoch 1/5
47/47 [==============================] - 5s 81ms/step - loss: 0.6359 - accuracy: 0.6173 - val_loss: 0.4454 - val_accuracy: 0.8180
Epoch 2/5
47/47 [==============================] - 5s 95ms/step - loss: 0.4556 - accuracy: 0.7980 - val_loss: 0.4481 - val_accuracy: 0.8020
Epoch 3/5
47/47 [==============================] - 4s 74ms/step - loss: 0.4090 - accuracy: 0.8213 - val_loss: 0.3351 - val_accuracy: 0.8600
Epoch 4/5
47/47 [==============================] - 4s 72ms/step - loss: 0.3708 - accuracy: 0.8480 - val_loss: 0.3151 - val_accuracy: 0.8820
Epoch 5/5
47/47 [==============================] - 5s 88ms/step - loss: 0.3423 - accuracy: 0.8527 - val_loss: 0.3156 - val_accuracy: 0.8580
